In [6]:
import pandas as pd
import numpy as np
import re
import tensorflow as tf
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from spellchecker import SpellChecker
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, TFBertForSequenceClassification
from tensorflow.keras.preprocessing.sequence import pad_sequences
import matplotlib.pyplot as plt

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import string
import re
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Flatten, Dense, Dropout, LSTM, Bidirectional
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from scikeras.wrappers import KerasClassifier, KerasRegressor
from sklearn.model_selection import GridSearchCV


# Load data
train_data = pd.read_csv('final_train.csv')
test_data = pd.read_csv('final_test.csv')


In [7]:
# Tokenize text
tokenizer = Tokenizer()
tokenizer.fit_on_texts(train_data['text'])

X_train = tokenizer.texts_to_sequences(train_data['text'])
X_test = tokenizer.texts_to_sequences(test_data['text'])

# Pad sequences
maxlen = 100  # adjust as needed
X_train = pad_sequences(X_train, padding='post', maxlen=maxlen)
X_test = pad_sequences(X_test, padding='post', maxlen=maxlen)

# Encode labels
label_mapping = {
    'bully-Spam': 0,
    'not_bully-Spam': 1,
    'bully-Ham': 2,
    'not_bully-Ham': 3
}
y_train = train_data['new_label'].map(label_mapping)
y_test = test_data['new_label'].map(label_mapping)

In [8]:
from keras.layers import Bidirectional

# Build Bidirectional LSTM model
model = Sequential()
model.add(Embedding(input_dim=len(tokenizer.word_index)+1, output_dim=50, input_length=maxlen))
model.add(Bidirectional(LSTM(64)))
model.add(Dense(4, activation='softmax'))

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train Bidirectional LSTM model
history=model.fit(X_train, y_train, epochs=15, batch_size=32, validation_split=0.1)


Epoch 1/15
40/40 [==============================] - 9s 104ms/step - loss: 1.1324 - accuracy: 0.4354 - val_loss: 1.1840 - val_accuracy: 0.4752
Epoch 2/15
40/40 [==============================] - 3s 71ms/step - loss: 0.8125 - accuracy: 0.6178 - val_loss: 0.7736 - val_accuracy: 0.8794
Epoch 3/15
40/40 [==============================] - 3s 73ms/step - loss: 0.6519 - accuracy: 0.8121 - val_loss: 0.6531 - val_accuracy: 0.8936
Epoch 4/15
40/40 [==============================] - 3s 72ms/step - loss: 0.4105 - accuracy: 0.8906 - val_loss: 0.6899 - val_accuracy: 0.7305
Epoch 5/15
40/40 [==============================] - 3s 71ms/step - loss: 0.2345 - accuracy: 0.9334 - val_loss: 0.5283 - val_accuracy: 0.8156
Epoch 6/15
40/40 [==============================] - 3s 71ms/step - loss: 0.1529 - accuracy: 0.9572 - val_loss: 0.6748 - val_accuracy: 0.8014
Epoch 7/15
40/40 [==============================] - 3s 70ms/step - loss: 0.1031 - accuracy: 0.9675 - val_loss: 1.1162 - val_accuracy: 0.6879
Epoch 8/15
4

In [9]:
# Evaluate Bi-LSTM model
from sklearn.metrics import classification_report
predictions = model.predict(X_test)
y_pred = predictions.argmax(axis=1)
print("Bi-LSTM Classification Report:")
print(classification_report(y_test, y_pred))

14/14 [==============================] - 1s 24ms/step
Bi-LSTM Classification Report:
              precision    recall  f1-score   support

           0       1.00      0.90      0.95        10
           1       0.91      0.83      0.87       115
           2       0.82      0.85      0.84       102
           3       0.85      0.87      0.86       199

    accuracy                           0.86       426
   macro avg       0.89      0.87      0.88       426
weighted avg       0.86      0.86      0.86       426

